## Summary

Analytical alchemical estimate of relative energy between N2 and BF with reference CO is performed.  
Analytical gradient w.r.t. nuclear coordinates is calculated to check the convergence of wavefunction.

The results of the alchemical estimate of $\mathrm{(N_2 - BF\textrm{ relative energy})_{alchemy}} - \mathrm{(N_2 - BF\textrm{ relative energy})_{SCF}} =\Delta E$ are  
| Included terms | $\Delta E$ (Hartree) |
| -- | -- |
| Hellmann-Feynman                  | 0.02037  |
| Hellmann-Feynman + 3rd derivative | -0.0057 |

For small CO, the mutation of two atomic species can be seen as a very large perturbation. Surprisingly, however, with the 3rd energy derivative, the error is of the order of mHa.

## Calculation

In [1]:
from pyscf import gto,scf
import pyscf
import numpy as np
import basis_set_exchange as bse

pyscf.__version__

'2.2.1'

In [2]:
from AP_class import APDFT_perturbator as AP

In [3]:
basis_set = bse.get_basis('pcX-2',fmt="nwchem")
# basis_set = "def2-TZVP"
dft_func = "pbe0"

In [4]:
mol_CO=gto.M(atom= "C 0 0 0; O 0 0 1.1",basis=basis_set)
print(mol_CO.atom_coords())
mf_co=scf.RKS(mol_CO)
mf_co.xc = dft_func
mf_co.scf()
mf_co.nuc_grad_method().kernel()
ap_co=AP(mf_co,sites=[0,1])

[[0.         0.         0.        ]
 [0.         0.         2.07869874]]
converged SCF energy = -113.230797527533
--------------- RKS gradients ---------------
         x                y                z
0 C    -0.0000000000    -0.0000000000     0.0602666334
1 O     0.0000000000     0.0000000000    -0.0602708853
----------------------------------------------


Calculate $\partial E/\partial Z_i$, hessian $\partial^2E/\partial Z_i\partial Z_j$ and cubic derivatives $\partial^3E/\partial Z_i\partial Z_j\partial Z_k$

In [5]:
print(ap_co.build_elec_gradient())
print(ap_co.build_elec_hessian())
print(ap_co.build_cubic())

[-18.5478735  -25.14633724]
[[-2.97157942 -0.02342743]
 [-0.02342743 -3.64448551]]
[[[-0.20456914  0.11836782]
  [ 0.11836782  0.08776343]]

 [[ 0.11836782  0.08776343]
  [ 0.08776343 -0.21809864]]]


### BF

APDFT

In [6]:
bf_apdft0 = ap_co.elec_APDFT0(np.asarray([-1,1]))
bf_apdft1 = ap_co.elec_APDFT1(np.asarray([-1,1]))
bf_apdft2 = ap_co.elec_APDFT2(np.asarray([-1,1]))
bf_apdft3 = ap_co.elec_APDFT3(np.asarray([-1,1]))

SCF

In [7]:
mol_BF=gto.M(atom= "B 0 0 0; F 0 0 1.1",basis=basis_set)
mf_bf=scf.RKS(mol_BF)
mf_bf.xc = dft_func
mf_bf.scf()
mf_bf.nuc_grad_method().kernel()
bf_energy = mf_bf.e_tot - mf_bf.energy_nuc()

converged SCF energy = -124.546122345499
--------------- RKS gradients ---------------
         x                y                z
0 B     0.0000000000    -0.0000000000     0.2857554094
1 F    -0.0000000000     0.0000000000    -0.2857954779
----------------------------------------------


### N2

APDFT

In [8]:
n2_apdft0 = ap_co.elec_APDFT0(np.asarray([1,-1]))
n2_apdft1 = ap_co.elec_APDFT1(np.asarray([1,-1]))
n2_apdft2 = ap_co.elec_APDFT2(np.asarray([1,-1]))
n2_apdft3 = ap_co.elec_APDFT3(np.asarray([1,-1]))
n2_apdft0 = ap_co.elec_APDFT0(np.asarray([1,-1]))

SCF

In [9]:
mol_NN=gto.M(atom= "N 0 0 0; N 0 0 1.1",basis=basis_set)
mf_nn=scf.RKS(mol_NN)
mf_nn.xc = dft_func
mf_nn.scf()
mf_nn.nuc_grad_method().kernel()
n2_energy = mf_nn.e_tot - mf_nn.energy_nuc()

converged SCF energy = -109.445281158139
--------------- RKS gradients ---------------
         x                y                z
0 N    -0.0000000000     0.0000000000    -0.0315355714
1 N     0.0000000000    -0.0000000000     0.0315355714
----------------------------------------------


Relative estimate of N2-BF energy

In [10]:
n2_bf_relative_apdft1 = ap_co.relative_elec_APDFT1(np.asarray([1,-1]))
n2_bf_relative_apdft3 = ap_co.relative_elec_APDFT3(np.asarray([1,-1]))

## Accuracy

BF

In [11]:
print(bf_apdft0 - bf_energy)
print(bf_apdft1 - bf_energy)
print(bf_apdft2 - bf_energy)
print(bf_apdft3 - bf_energy)

9.872114242729907
3.2736505043952775
-0.010954534989878084
0.0020927459560482475


N2

In [12]:
print(n2_apdft0 - n2_energy)
print(n2_apdft1 - n2_energy)
print(n2_apdft2 - n2_energy)
print(n2_apdft3 - n2_energy)

-3.3044461776488276
3.294017560685802
0.009412521300646404
-0.003634759645279928


N2 - BF relative energy

In [13]:
print("SCF")
print(n2_energy - bf_energy)
print()
print("APDFT")
print("APDFT0:", n2_apdft0 - bf_apdft0)
print("APDFT1:", n2_apdft1 - bf_apdft1)
print("APDFT2:", n2_apdft2 - bf_apdft2)
print("APDFT3:", n2_apdft3 - bf_apdft3)
print()
print("Relative alchemical estimate")
print("Hellmann-Feynman                 :", n2_bf_relative_apdft1)
print("Hellmann-Feynman + 3rd derivative:", n2_bf_relative_apdft3)

SCF
13.176560420378735

APDFT
APDFT0: 0.0
APDFT1: 13.19692747666926
APDFT2: 13.19692747666926
APDFT3: 13.170832914777407

Relative alchemical estimate
Hellmann-Feynman                 : 13.19692747666923
Hellmann-Feynman + 3rd derivative: 13.170832914777368


$\mathrm{(N_2 - BF\textrm{ relative energy})_{alchemy}} - \mathrm{(N_2 - BF\textrm{ relative energy})_{SCF}}$

In [14]:
print("APDFT")
print("APDFT0:", n2_apdft0 - bf_apdft0 - (n2_energy - bf_energy))
print("APDFT1:", n2_apdft1 - bf_apdft1 - (n2_energy - bf_energy))
print("APDFT2:", n2_apdft2 - bf_apdft2 - (n2_energy - bf_energy))
print("APDFT3:", n2_apdft3 - bf_apdft3 - (n2_energy - bf_energy))
print()
print("Relative alchemical estimate")
print("Hellmann-Feynman                 :", n2_bf_relative_apdft1 - (n2_energy - bf_energy))
print("Hellmann-Feynman + 3rd derivative:", n2_bf_relative_apdft3 - (n2_energy - bf_energy))

APDFT
APDFT0: -13.176560420378735
APDFT1: 0.020367056290524488
APDFT2: 0.020367056290524488
APDFT3: -0.0057275056013281755

Relative alchemical estimate
Hellmann-Feynman                 : 0.020367056290496066
Hellmann-Feynman + 3rd derivative: -0.005727505601367255
